# Comparing TIAToolbox Nucleus Segmentation with IDC DICOM Segmentations

<a href="https://colab.research.google.com/github/fedorov/idc-tiatoolbox/blob/main/notebooks/08_comparing_with_idc_segmentations_copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

IDC hosts nucleus segmentations stored as **DICOM Segmentation (SEG)** objects — binary pixel masks representing segmented nuclei on whole slide images. Unlike DICOM ANN (which stores point/polygon annotations), SEG objects contain actual pixel-level masks.

In this notebook, we:
1. Find a TCGA slide that has nucleus segmentations (DICOM SEG) in IDC
2. Download both the slide and its segmentation
3. Parse the DICOM SEG using `highdicom` to extract binary masks
4. Run TIAToolbox's HoVer-Net on the same region
5. Compare the two sets of results using pixel-level metrics (IoU, Dice)

**GPU recommended** for HoVer-Net inference.

## Installation

Run the cell below to install dependencies. **On Colab, the runtime will automatically restart** after installation. After the restart, continue from the imports cell below.

In [ ]:
%pip install tiatoolbox idc-index openslide-bin "numcodecs<0.16" highdicom wsidicom shapely opencv-python

# Restart runtime to pick up updated numpy (required on Colab)
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import os
import joblib
import pydicom
import highdicom as hd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import cv2
import torch
from PIL import Image

from idc_index import IDCClient
from tiatoolbox.wsicore.wsireader import WSIReader
from tiatoolbox.models.engine.nucleus_instance_segmentor import NucleusInstanceSegmentor

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    try:
        torch.zeros(1, device="cuda")
    except RuntimeError:
        print("CUDA available but not functional — falling back to CPU")
        device = "cpu"
print(f"Using device: {device}")
if device == "cpu":
    print("Note: GPU is recommended. In Colab: Runtime > Change runtime type > T4 GPU")

## 1. Find a Slide with DICOM Segmentations

DICOM Segmentations (SEG) on slide microscopy images are indexed in `seg_index`. We join with `index` to filter by source modality (SM) and find nucleus-related segmentations.

In [ ]:
idc_client = IDCClient()
idc_client.fetch_index("sm_index")
idc_client.fetch_index("seg_index")

In [ ]:
# Find DICOM SEG series that segment slide microscopy images
sm_segs = idc_client.sql_query("""
    SELECT
        seg.SeriesInstanceUID as seg_series_uid,
        seg.segmented_SeriesInstanceUID as slide_series_uid,
        seg.AlgorithmName,
        seg.total_segments,
        i_seg.collection_id as seg_collection,
        i_seg.analysis_result_id,
        i_slide.collection_id as slide_collection,
        i_slide.PatientID,
        ROUND(i_slide.series_size_MB, 1) as slide_size_mb,
        ROUND(i_seg.series_size_MB, 1) as seg_size_mb,
        s.ObjectiveLensPower,
        s.min_PixelSpacing_2sf as pixel_spacing_mm
    FROM seg_index seg
    JOIN index i_seg ON seg.SeriesInstanceUID = i_seg.SeriesInstanceUID
    JOIN index i_slide ON seg.segmented_SeriesInstanceUID = i_slide.SeriesInstanceUID
    JOIN sm_index s ON seg.segmented_SeriesInstanceUID = s.SeriesInstanceUID
    WHERE i_slide.Modality = 'SM'
        AND s.ObjectiveLensPower >= 20
    ORDER BY i_slide.series_size_MB ASC
    LIMIT 20
""")

print(f"Found {len(sm_segs)} slides with DICOM Segmentations")
sm_segs

In [ ]:
# Show available algorithms and analysis results
print("Available segmentation sources:")
print(sm_segs[['AlgorithmName', 'analysis_result_id', 'slide_collection']].drop_duplicates())

In [ ]:
# Select a slide - prefer smaller ones for faster demo
# Filter for nucleus-related segmentations if possible
nucleus_segs = sm_segs[
    sm_segs['AlgorithmName'].str.lower().str.contains('nucle|hover|cell|instance', na=False) |
    sm_segs['analysis_result_id'].str.lower().str.contains('nucle|seg', na=False)
]

if len(nucleus_segs) > 0:
    selected = nucleus_segs.iloc[0]
    print("Found nucleus-related segmentation")
else:
    selected = sm_segs.iloc[0]
    print("Using first available segmentation")

slide_series_uid = selected['slide_series_uid']
seg_series_uid = selected['seg_series_uid']

print(f"\nSelected slide:")
print(f"  Patient: {selected['PatientID']}")
print(f"  Collection: {selected['slide_collection']}")
print(f"  Slide: {slide_series_uid} ({selected['slide_size_mb']} MB)")
print(f"  SEG: {seg_series_uid} ({selected['seg_size_mb']} MB)")
print(f"  Algorithm: {selected['AlgorithmName']}")
print(f"  Segments: {selected['total_segments']}")
print(f"  Objective: {selected['ObjectiveLensPower']}x")

## 2. Download the Slide and its Segmentation

In [ ]:
download_dir = './slides'
seg_dir = './segmentations'
os.makedirs(download_dir, exist_ok=True)
os.makedirs(seg_dir, exist_ok=True)

# Download the slide
idc_client.download_from_selection(
    downloadDir=download_dir,
    seriesInstanceUID=[slide_series_uid],
    dirTemplate='%SeriesInstanceUID'
)

# Download the SEG object (flat, no directory template)
idc_client.download_from_selection(
    downloadDir=seg_dir,
    seriesInstanceUID=[seg_series_uid],
    dirTemplate=None
)

slide_path = os.path.join(download_dir, slide_series_uid)
seg_files = [f for f in os.listdir(seg_dir) if f.endswith('.dcm')]
print(f"Downloaded slide: {slide_path}")
print(f"Downloaded {len(seg_files)} SEG file(s)")

In [ ]:
# Open slide with TIAToolbox
reader = WSIReader.open(slide_path)

# DICOMWSIReader may not populate objective_power or mpp
info = reader.info
if info.objective_power is None:
    info.objective_power = float(selected['ObjectiveLensPower'])
if info.mpp is None:
    pixel_spacing_um = float(selected['pixel_spacing_mm']) * 1000
    info.mpp = np.array([pixel_spacing_um, pixel_spacing_um])

print(f"Slide: {type(reader).__name__}, dimensions: {info.slide_dimensions}")
print(f"MPP: {info.mpp}")

thumbnail = reader.slide_thumbnail(resolution=1.25, units="power")
plt.figure(figsize=(10, 8))
plt.imshow(thumbnail)
plt.title(f"Slide Thumbnail ({selected['PatientID']})", fontsize=14)
plt.axis('off')
plt.show()

## 3. Parse IDC DICOM SEG

DICOM Segmentation objects store binary pixel masks. We use `highdicom` to parse them and extract the segmentation mask.

In [ ]:
# Load the SEG DICOM object
seg_path = os.path.join(seg_dir, seg_files[0])
seg_dcm = pydicom.dcmread(seg_path)

print(f"SEG SOP Class: {seg_dcm.SOPClassUID}")
print(f"Rows: {seg_dcm.Rows}, Columns: {seg_dcm.Columns}")
print(f"Number of Frames: {getattr(seg_dcm, 'NumberOfFrames', 1)}")

# List segments
print(f"\nSegments:")
for i, segment in enumerate(seg_dcm.SegmentSequence):
    print(f"  {i+1}: {segment.SegmentLabel} (Algorithm: {getattr(segment, 'SegmentAlgorithmName', 'N/A')})")

In [ ]:
# Use highdicom to read segmentation
seg = hd.seg.segread(seg_path)

print(f"Segmentation type: {type(seg).__name__}")
print(f"Number of segments: {len(seg.SegmentSequence)}")

# Get frame positions to understand spatial coverage
if hasattr(seg, 'PerFrameFunctionalGroupsSequence'):
    n_frames = len(seg.PerFrameFunctionalGroupsSequence)
    print(f"Number of frames: {n_frames}")

In [ ]:
# Extract pixel data for the first segment
# Note: SEG pixel data structure varies - it may be tiled or contain multiple frames
try:
    # Try highdicom's get_pixels_by_segment for single-frame or simple cases
    seg_pixels = seg.get_pixels_by_segment(segment_numbers=[1])
    print(f"Extracted segmentation shape: {seg_pixels.shape}")
except Exception as e:
    print(f"Using raw pixel array: {e}")
    # Fall back to raw pixel array
    seg_pixels = seg.pixel_array
    print(f"Raw pixel array shape: {seg_pixels.shape}")

In [ ]:
# Get spatial information from the SEG to understand coverage
# Extract position of first frame
if hasattr(seg, 'PerFrameFunctionalGroupsSequence') and len(seg.PerFrameFunctionalGroupsSequence) > 0:
    first_frame = seg.PerFrameFunctionalGroupsSequence[0]
    if hasattr(first_frame, 'PlanePositionSlideSequence'):
        pos = first_frame.PlanePositionSlideSequence[0]
        row_pos = float(pos.RowPositionInTotalPixelMatrix)
        col_pos = float(pos.ColumnPositionInTotalPixelMatrix)
        print(f"First frame position: row={row_pos}, col={col_pos}")
    
    # Get pixel spacing from SEG
    if hasattr(first_frame, 'PixelMeasuresSequence'):
        pm = first_frame.PixelMeasuresSequence[0]
        seg_pixel_spacing = float(pm.PixelSpacing[0])
        print(f"SEG pixel spacing: {seg_pixel_spacing} mm")

## 4. Extract Matching Region from Slide

We need to extract the same region from the slide that the SEG covers. SEG frame positions tell us where each frame maps to in the source image.

In [ ]:
# Get pixel spacing from slide for coordinate conversion
idc_client.fetch_index("sm_instance_index")

pixel_info = idc_client.sql_query(f"""
    SELECT
        TotalPixelMatrixColumns as width,
        TotalPixelMatrixRows as height,
        PixelSpacing_0 as pixel_spacing_mm
    FROM sm_instance_index
    WHERE SeriesInstanceUID = '{slide_series_uid}'
    ORDER BY TotalPixelMatrixColumns DESC
    LIMIT 1
""")

slide_px_spacing = pixel_info.iloc[0]['pixel_spacing_mm']
slide_width = int(pixel_info.iloc[0]['width'])
slide_height = int(pixel_info.iloc[0]['height'])
print(f"Slide pixel spacing: {slide_px_spacing:.6f} mm")
print(f"Full resolution dimensions: {slide_width} x {slide_height}")

In [ ]:
# Determine region covered by SEG
# For multi-frame SEG, calculate bounding box of all frames
if hasattr(seg, 'PerFrameFunctionalGroupsSequence'):
    rows_list = []
    cols_list = []
    for frame in seg.PerFrameFunctionalGroupsSequence:
        if hasattr(frame, 'PlanePositionSlideSequence'):
            pos = frame.PlanePositionSlideSequence[0]
            rows_list.append(float(pos.RowPositionInTotalPixelMatrix))
            cols_list.append(float(pos.ColumnPositionInTotalPixelMatrix))
    
    if rows_list:
        min_row, max_row = min(rows_list), max(rows_list)
        min_col, max_col = min(cols_list), max(cols_list)
        # Add frame dimensions
        seg_rows = seg.Rows
        seg_cols = seg.Columns
        
        # Bounds in slide pixel coordinates
        bounds_x = int(min_col) - 1  # DICOM uses 1-based indexing
        bounds_y = int(min_row) - 1
        bounds_w = int(max_col - min_col) + seg_cols
        bounds_h = int(max_row - min_row) + seg_rows
        
        print(f"SEG coverage: ({bounds_x}, {bounds_y}) to ({bounds_x + bounds_w}, {bounds_y + bounds_h})")
        print(f"Coverage size: {bounds_w} x {bounds_h} pixels")
else:
    # Single frame - use full SEG dimensions
    bounds_x, bounds_y = 0, 0
    bounds_w, bounds_h = seg.Columns, seg.Rows
    print(f"Single frame SEG: {bounds_w} x {bounds_h} pixels")

In [ ]:
# If coverage is too large, select a smaller region for comparison
max_tile_size = 2048

if bounds_w > max_tile_size or bounds_h > max_tile_size:
    # Center crop to max_tile_size
    center_x = bounds_x + bounds_w // 2
    center_y = bounds_y + bounds_h // 2
    
    tile_bounds = (
        max(0, center_x - max_tile_size // 2),
        max(0, center_y - max_tile_size // 2),
        min(slide_width, center_x + max_tile_size // 2),
        min(slide_height, center_y + max_tile_size // 2),
    )
    print(f"Cropping to {max_tile_size}x{max_tile_size} region")
else:
    tile_bounds = (
        max(0, bounds_x),
        max(0, bounds_y),
        min(slide_width, bounds_x + bounds_w),
        min(slide_height, bounds_y + bounds_h),
    )

print(f"Tile bounds: {tile_bounds}")
tile_w = tile_bounds[2] - tile_bounds[0]
tile_h = tile_bounds[3] - tile_bounds[1]
print(f"Tile size: {tile_w} x {tile_h}")

In [ ]:
# Extract tile from slide
# Read at native resolution to avoid DICOMWSIReader coordinate issues
tile = reader.read_bounds(
    bounds=tile_bounds,
    resolution=info.objective_power,
    units="power",
)

# Resize to expected size if needed
if tile.shape[0] != tile_h or tile.shape[1] != tile_w:
    tile = np.array(Image.fromarray(tile).resize((tile_w, tile_h), Image.LANCZOS))

print(f"Extracted tile shape: {tile.shape}")

plt.figure(figsize=(10, 10))
plt.imshow(tile)
plt.title(f"Extracted Region ({tile_w}x{tile_h})")
plt.axis('off')
plt.show()

## 5. Reconstruct IDC Segmentation Mask for the Region

We need to assemble the SEG frames that fall within our tile region into a single binary mask.

In [ ]:
# Create empty mask for the tile region
idc_mask = np.zeros((tile_h, tile_w), dtype=np.uint8)

# Get frame data and positions
if hasattr(seg, 'PerFrameFunctionalGroupsSequence'):
    pixel_array = seg.pixel_array
    
    for i, frame in enumerate(seg.PerFrameFunctionalGroupsSequence):
        if hasattr(frame, 'PlanePositionSlideSequence'):
            pos = frame.PlanePositionSlideSequence[0]
            frame_row = int(float(pos.RowPositionInTotalPixelMatrix)) - 1  # 1-based to 0-based
            frame_col = int(float(pos.ColumnPositionInTotalPixelMatrix)) - 1
            
            # Check if frame overlaps with our tile
            frame_right = frame_col + seg.Columns
            frame_bottom = frame_row + seg.Rows
            
            if (frame_col < tile_bounds[2] and frame_right > tile_bounds[0] and
                frame_row < tile_bounds[3] and frame_bottom > tile_bounds[1]):
                
                # Get frame data
                if pixel_array.ndim == 2:
                    frame_data = pixel_array if i == 0 else pixel_array
                else:
                    frame_data = pixel_array[i]
                
                # Calculate overlap region
                src_x1 = max(0, tile_bounds[0] - frame_col)
                src_y1 = max(0, tile_bounds[1] - frame_row)
                src_x2 = min(seg.Columns, tile_bounds[2] - frame_col)
                src_y2 = min(seg.Rows, tile_bounds[3] - frame_row)
                
                dst_x1 = max(0, frame_col - tile_bounds[0])
                dst_y1 = max(0, frame_row - tile_bounds[1])
                dst_x2 = dst_x1 + (src_x2 - src_x1)
                dst_y2 = dst_y1 + (src_y2 - src_y1)
                
                # Copy frame data to mask
                idc_mask[dst_y1:dst_y2, dst_x1:dst_x2] = np.maximum(
                    idc_mask[dst_y1:dst_y2, dst_x1:dst_x2],
                    frame_data[src_y1:src_y2, src_x1:src_x2]
                )
else:
    # Single frame case
    idc_mask = seg.pixel_array.astype(np.uint8)

print(f"IDC mask shape: {idc_mask.shape}")
print(f"IDC mask coverage: {np.sum(idc_mask > 0)} pixels ({100*np.sum(idc_mask > 0)/(tile_w*tile_h):.2f}%)")

In [ ]:
# Visualize IDC segmentation mask
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(tile)
axes[0].set_title("Original Tile")
axes[0].axis('off')

axes[1].imshow(idc_mask, cmap='gray')
axes[1].set_title("IDC Segmentation Mask")
axes[1].axis('off')

# Overlay
overlay = tile.copy()
overlay[idc_mask > 0] = [255, 0, 255]  # Magenta overlay
axes[2].imshow(overlay)
axes[2].set_title("IDC Segmentation Overlay")
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 6. Run TIAToolbox HoVer-Net on the Same Region

In [ ]:
# Save tile for HoVer-Net
tile_path = './tile_for_seg_comparison.png'
Image.fromarray(tile).save(tile_path)

# Run HoVer-Net
segmentor = NucleusInstanceSegmentor(
    pretrained_model="hovernet_fast-pannuke",
    num_loader_workers=0,
    num_postproc_workers=0,
    batch_size=8,
)

output = segmentor.predict(
    imgs=[tile_path],
    mode="tile",
    save_dir="./seg_comparison_results/",
    resolution=1.0,
    units="baseline",
    device=device,
)

print("HoVer-Net inference complete!")

In [ ]:
# Load HoVer-Net results
hovernet_nuclei = joblib.load(output[0][1] + '.dat')

print(f"TIAToolbox HoVer-Net detected: {len(hovernet_nuclei)} nuclei")

In [ ]:
# Convert HoVer-Net contours to binary mask
hovernet_mask = np.zeros((tile_h, tile_w), dtype=np.uint8)

for nuc_id, nuc_data in hovernet_nuclei.items():
    contour = nuc_data['contour'].astype(np.int32)
    cv2.fillPoly(hovernet_mask, [contour], 1)

print(f"HoVer-Net mask shape: {hovernet_mask.shape}")
print(f"HoVer-Net mask coverage: {np.sum(hovernet_mask > 0)} pixels ({100*np.sum(hovernet_mask > 0)/(tile_w*tile_h):.2f}%)")

In [ ]:
# Visualize HoVer-Net segmentation mask
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(tile)
axes[0].set_title("Original Tile")
axes[0].axis('off')

axes[1].imshow(hovernet_mask, cmap='gray')
axes[1].set_title("HoVer-Net Mask")
axes[1].axis('off')

# Overlay
overlay = tile.copy()
overlay[hovernet_mask > 0] = [0, 255, 255]  # Cyan overlay
axes[2].imshow(overlay)
axes[2].set_title("HoVer-Net Overlay")
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 7. Binary Mask Comparison: IoU and Dice

Now we compare the two binary masks using pixel-level metrics.

In [ ]:
# Ensure masks are binary
idc_binary = (idc_mask > 0).astype(np.uint8)
hovernet_binary = (hovernet_mask > 0).astype(np.uint8)

# Compute metrics
intersection = np.sum(idc_binary & hovernet_binary)
union = np.sum(idc_binary | hovernet_binary)
idc_sum = np.sum(idc_binary)
hovernet_sum = np.sum(hovernet_binary)

# IoU (Intersection over Union)
iou = intersection / union if union > 0 else 0

# Dice coefficient
dice = 2 * intersection / (idc_sum + hovernet_sum) if (idc_sum + hovernet_sum) > 0 else 0

print("Binary Mask Comparison Metrics")
print("=" * 40)
print(f"IDC mask pixels:      {idc_sum:,}")
print(f"HoVer-Net mask pixels: {hovernet_sum:,}")
print(f"Intersection:          {intersection:,}")
print(f"Union:                 {union:,}")
print(f"")
print(f"IoU (Jaccard):         {iou:.4f}")
print(f"Dice (F1):             {dice:.4f}")

In [ ]:
# Confusion matrix visualization
# True Positive (both agree) = green
# False Positive (HoVer-Net only) = red
# False Negative (IDC only) = blue

tp = idc_binary & hovernet_binary
fp = (~idc_binary.astype(bool)) & hovernet_binary.astype(bool)
fn = idc_binary.astype(bool) & (~hovernet_binary.astype(bool))

# Create color-coded overlay
confusion_overlay = tile.copy().astype(np.float32)
confusion_overlay[tp > 0] = [0, 255, 0]    # Green - agreement
confusion_overlay[fp > 0] = [255, 0, 0]    # Red - HoVer-Net only
confusion_overlay[fn > 0] = [0, 0, 255]    # Blue - IDC only
confusion_overlay = confusion_overlay.astype(np.uint8)

print(f"True Positives (agreement):   {np.sum(tp):,} pixels")
print(f"False Positives (TIA only):   {np.sum(fp):,} pixels")
print(f"False Negatives (IDC only):   {np.sum(fn):,} pixels")

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 4, figsize=(24, 6))

axes[0].imshow(tile)
axes[0].set_title("Original Tile", fontsize=13)
axes[0].axis('off')

# IDC mask overlay
idc_overlay = tile.copy()
idc_overlay[idc_binary > 0] = [255, 0, 255]
axes[1].imshow(idc_overlay)
axes[1].set_title(f"IDC SEG\n({idc_sum:,} pixels)", fontsize=13)
axes[1].axis('off')

# HoVer-Net mask overlay
hovernet_overlay = tile.copy()
hovernet_overlay[hovernet_binary > 0] = [0, 255, 255]
axes[2].imshow(hovernet_overlay)
axes[2].set_title(f"TIAToolbox HoVer-Net\n({hovernet_sum:,} pixels)", fontsize=13)
axes[2].axis('off')

# Confusion overlay
axes[3].imshow(confusion_overlay)
axes[3].set_title(f"Comparison (IoU={iou:.3f}, Dice={dice:.3f})\nGreen=Both, Red=TIA only, Blue=IDC only", fontsize=11)
axes[3].axis('off')

plt.tight_layout()
plt.show()

### Zoomed Comparison

In [ ]:
# Zoom into a 512x512 region with nuclei
zoom_size = 512

# Find region with nuclei
combined_mask = idc_binary | hovernet_binary
if np.sum(combined_mask) > 0:
    # Find centroid of nuclei
    coords = np.argwhere(combined_mask)
    center_y, center_x = coords.mean(axis=0).astype(int)
else:
    center_x, center_y = tile_w // 2, tile_h // 2

zoom_x = max(0, min(tile_w - zoom_size, center_x - zoom_size // 2))
zoom_y = max(0, min(tile_h - zoom_size, center_y - zoom_size // 2))

# Extract zoomed regions
zoomed_tile = tile[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]
zoomed_confusion = confusion_overlay[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]
zoomed_idc = idc_overlay[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]
zoomed_hovernet = hovernet_overlay[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(zoomed_tile)
axes[0].set_title("Original (zoomed)", fontsize=12)
axes[0].axis('off')

axes[1].imshow(zoomed_idc)
axes[1].set_title("IDC SEG (zoomed)", fontsize=12)
axes[1].axis('off')

axes[2].imshow(zoomed_hovernet)
axes[2].set_title("HoVer-Net (zoomed)", fontsize=12)
axes[2].axis('off')

axes[3].imshow(zoomed_confusion)
axes[3].set_title("Comparison (zoomed)", fontsize=12)
axes[3].axis('off')

plt.tight_layout()
plt.show()

## Summary

In this notebook, we learned how to:

- Use `seg_index` to find IDC's **DICOM Segmentation** objects on slide microscopy images
- Download both the slide and its DICOM SEG
- Parse DICOM SEG objects using `highdicom` to extract binary masks
- Reconstruct segmentation masks from multi-frame DICOM SEG
- Run TIAToolbox's **HoVer-Net** and convert instance contours to binary masks
- **Compare using pixel-level metrics**: IoU (Jaccard Index) and Dice coefficient
- **Visualize** agreement and disagreement with color-coded overlays

**Key observations:**
- IoU and Dice provide quantitative measures of segmentation overlap
- Color-coded overlays show where methods agree (green), HoVer-Net-only detections (red), and IDC-only (blue)
- Differences may arise from different algorithms, thresholds, or training data
- Both approaches are complementary: IDC provides pre-computed segmentations, while TIAToolbox enables custom analysis

## Acknowledgments

- **IDC:** Fedorov, A., et al. "National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence." *RadioGraphics* 43.12 (2023). https://doi.org/10.1148/rg.230180
- **TIAToolbox:** Pocock, J., et al. "TIAToolbox as an end-to-end library for advanced tissue image analytics." *Communications Medicine* 2, 120 (2022). https://doi.org/10.1038/s43856-022-00186-5
- **HoVer-Net:** Graham, S., et al. "Hover-Net: Simultaneous segmentation and classification of nuclei in multi-tissue histology images." *Medical Image Analysis* 58 (2019). https://doi.org/10.1016/j.media.2019.101563
- **highdicom:** Bridge, C., et al. "Highdicom: A Python library for standardized encoding of image annotations and machine learning model outputs in pathology and radiology." *Journal of Digital Imaging* (2022). https://doi.org/10.1007/s10278-022-00683-y